In [1]:
import os, math, import_ipynb, cv2, dlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing import image
from utils import *

Using TensorFlow backend.
/Users/kenyew/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kenyew/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kenyew/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kenyew/anaconda3/lib/python3.7/site-packages/t

importing Jupyter notebook from utils.ipynb


/Users/kenyew/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/kenyew/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/kenyew/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/kenyew/anaconda3/lib/python3.7/site-packages/tensor

In [2]:
os.chdir("/Users/kenyew/Documents/Git/AMLSassignment19_20/AMLS_19-20_Ken_Yew_Piong_SN16004444/")
os.getcwd()

'/Users/kenyew/Documents/Git/AMLSassignment19_20/AMLS_19-20_Ken_Yew_Piong_SN16004444'

In [3]:
df = pd.read_csv("./Datasets/celeba/labels.csv")
df = split_df(df)
print(df)

      img_name gender smiling
0        0.jpg     -1       1
1        1.jpg     -1       1
2        2.jpg      1      -1
3        3.jpg     -1      -1
4        4.jpg     -1      -1
...        ...    ...     ...
4995  4995.jpg      1       1
4996  4996.jpg      1       1
4997  4997.jpg      1       1
4998  4998.jpg      1       1
4999  4999.jpg      1       1

[5000 rows x 3 columns]


In [6]:
basedir = './Datasets/celeba'
images_dir = os.path.join(basedir,'img')
labels_filename = 'labels.csv'
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((shape.num_parts, 2), dtype=dtype)

    # loop over all facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, shape.num_parts):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

def run_dlib_shape(image):
    # in this function we load the image, detect the landmarks of the face, and then return the image and the landmarks
    # load the input image, resize it, and convert it to grayscale
    resized_image = image.astype('uint8')

    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    gray = gray.astype('uint8')

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    num_faces = len(rects)

    if num_faces == 0:
        return None, resized_image

    face_areas = np.zeros((1, num_faces))
    face_shapes = np.zeros((136, num_faces), dtype=np.int64)

    # loop over the face detections
    for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the facial landmark (x, y)-coordinates to a NumPy
        # array
        temp_shape = predictor(gray, rect)
        temp_shape = shape_to_np(temp_shape)

        # convert dlib's rectangle to a OpenCV-style bounding box
        # [i.e., (x, y, w, h)],
        #   (x, y, w, h) = face_utils.rect_to_bb(rect)
        (x, y, w, h) = rect_to_bb(rect)
        face_shapes[:, i] = np.reshape(temp_shape, [136])
        face_areas[0, i] = w * h
    # find largest face and keep
    dlibout = np.reshape(np.transpose(face_shapes[:, np.argmax(face_areas)]), [68, 2])

    return dlibout, resized_image

"""
Extract Features Function
"""
image_paths = [os.path.join(images_dir, l) for l in os.listdir(images_dir)]
target_size = None
labels_file = open(os.path.join(basedir, labels_filename), 'r')
lines = labels_file.readlines()

# img_name_labels = {line.split('\t')[0] : line.split('\t')[1] for line in lines[1:]}
gender_labels = {line.split('\t')[0] : int(line.split('\t')[2]) for line in lines[1:]}
# smiling_labels = {line.split('\t')[0] : int(line.split('\t')[3]) for line in lines[1:]}

# file_name= image_paths[0].split('.')[1].split('/')[-1]
    
if os.path.isdir(images_dir):
    all_features = []
    all_labels = []
    for img_path in image_paths:
        file_name= img_path.split('.')[1].split('/')[-1]

        # load image
        img = image.img_to_array(
            image.load_img(img_path,
                           target_size=target_size,
                           interpolation='bicubic'))
        features, _ = run_dlib_shape(img)
        if features is not None:
            all_features.append(features)
            all_labels.append(gender_labels[file_name])

landmark_features = np.array(all_features)
gender_labels = (np.array(all_labels) + 1)/2 # simply converts the -1 into 0, so male=0 and female=1
print(type(landmark_features),':',landmark_features)
print(type(gender_labels),':',gender_labels)

KeyError: '63'

In [ ]:

# gender_labels = {line.split(',')[0] : int(line.split(',')[6]) for line in lines[2:]}
# if os.path.isdir(images_dir):
#     all_features = []
#     all_labels = []
#     for img_path in image_paths:
#         file_name= img_path.split('.')[1].split('/')[-1]

#         # load image
#         img = image.img_to_array(
#             image.load_img(img_path,
#                            target_size=target_size,
#                            interpolation='bicubic'))
#         features, _ = run_dlib_shape(img)
#         if features is not None:
#             all_features.append(features)
#             all_labels.append(gender_labels[file_name])

# landmark_features = np.array(all_features)
# gender_labels = (np.array(all_labels) + 1)/2 # simply converts the -1 into 0, so male=0 and female=1
# return landmark_features, gender_labels